In [ ]:
import MySQLdb
from pandas.io import sql

conn = MySQLdb.connect(host="localhost", user="root", passwd="MYSQLROOTPASSWORD", db="openmrs") 
openmrscursor = conn.cursor() 

In [ ]:
import pandas as pd
import numpy as np

df_obs = pd.read_sql('select concept_id, location_id from `obs`', con = conn)
df_obs

In [ ]:
mimicconn = MySQLdb.connect(host="localhost", user="root", passwd="MYSQLROOTPASSWORD", db="mimic4") 
mimiccursor = mimicconn.cursor()

In [ ]:
df_labevents_mimic_coded = pd.read_sql("SELECT subject_id, value, hadm_id, itemid, storetime from mimic4.labevents where hadm_id IS NOT NULL AND value IS NOT NULL AND value regexp '[[:alpha:]]'", con = mimicconn)
df_labevents_mimic_coded

In [ ]:
df_labitems_mimic_ = pd.read_sql("SELECT itemid, label from mimic4.d_labitems where itemid IS NOT NULL AND label IS NOT NULL", con = mimicconn)

In [ ]:
df_labitems_mimic_

In [ ]:
df_labitems_mimic_['label'] = df_labitems_mimic_['label'].str.lower()

In [ ]:
dict_ = dict(zip(df_labitems_mimic_.itemid, df_labitems_mimic_.label))
dict_

In [ ]:
df_labevents_mimic_coded['label'] = df_labevents_mimic_coded['itemid'].map(dict_)
df_labevents_mimic_coded

In [ ]:
df_labevents_mimic_coded.dtypes

In [ ]:
df_labevents_mimic_coded.isnull().sum()

In [ ]:
_df_ = pd.read_csv('non-numeric-labevents_.csv')

_df_= _df_[['value', 'value1']]
_df_.dropna(inplace = True)
_df_

In [ ]:
_df_ = _df_.rename(columns={'value1':'value_text'})   
_df_

In [ ]:
_df_['value'] = _df_['value'].str.lower()
_df_['value_text'] = _df_['value_text'].str.lower()

_df_

In [ ]:
df_labevents_mimic_coded['value'] = df_labevents_mimic_coded['value'].str.lower()
df_labevents_mimic_coded

In [ ]:
dict_values = dict(zip(_df_.value, _df_.value_text))
dict_values

In [ ]:
df_labevents_mimic_coded['value'] = df_labevents_mimic_coded['value'].map(dict_values)
df_labevents_mimic_coded

In [ ]:
concept_lab_coded = pd.read_sql('select * from concept where class_id = 1', con=conn)
concept_lab_coded

In [ ]:
dict_concept_coded = dict(zip(concept_lab_coded.description, concept_lab_coded.concept_id))
len(dict_concept_coded)

In [ ]:
df_labevents_mimic_coded['concept_id'] = df_labevents_mimic_coded['label'].map(dict_concept_coded)
df_labevents_mimic_coded

In [ ]:
df_labevents_mimic_coded = df_labevents_mimic_coded.rename({"subject_id": "person_id", "hadm_id": "encounter_id", "storetime": "obs_datetime", "value":"value_text" }, axis='columns')

df_labevents_mimic_coded = df_labevents_mimic_coded.drop(columns=['itemid', 'label'])
df_labevents_mimic_coded['date_created']= '2022-01-01'
import uuid
df_labevents_mimic_coded['uuid'] = [uuid.uuid4() for x in range(len(df_labevents_mimic_coded))]
df_labevents_mimic_coded['creator'] = 1
df_labevents_mimic_coded

In [ ]:
df_labevents_mimic_coded.isnull().sum()

In [ ]:
conceptid_answer = pd.read_sql('select * from concept where class_id = 11 and datatype_id=4', con=conn)
conceptid_answer['description'] = conceptid_answer['description'].str.lower()

In [ ]:
df_labevents_mimic_coded_copy = df_labevents_mimic_coded.copy()


In [ ]:
dict_answer_conceptid = dict(zip(conceptid_answer.description, conceptid_answer.concept_id))
len(dict_answer_conceptid)

In [ ]:
df_labevents_mimic_coded['value_coded'] = df_labevents_mimic_coded['value_text'].map(dict_answer_conceptid)
df_labevents_mimic_coded

In [ ]:
df_labevents_mimic_coded.value_text = df_labevents_mimic_coded.value_text.str.replace('[^a-zA-Z]', '')

In [ ]:
df_labevents_mimic_coded = df_labevents_mimic_coded[~df_labevents_mimic_coded['value_text'].str.isdigit()]

In [ ]:
df_labevents_mimic_coded['value_text'] = df_labevents_mimic_coded.value_text.replace('',np.nan,regex = True)

In [ ]:
df_labevents_mimic_coded=df_labevents_mimic_coded.dropna(subset=['value_text'])

In [ ]:
df_labevents_mimic_coded['value_coded'] = df_labevents_mimic_coded['value_coded'].astype(int)

In [ ]:
df_labevents_mimic_coded = df_labevents_mimic_coded.drop(columns= ['value_text'])

In [ ]:
df_labevents_mimic_coded

In [ ]:
df_labevents_mimic_coded = df_labevents_mimic_coded.replace({np.NAN: None})

In [ ]:
column_names = ['person_id', 'concept_id', 'encounter_id', 'obs_datetime', 'value_coded', 'creator', 'date_created', 'uuid']
df_labevents_mimic_coded = df_labevents_mimic_coded.reindex(columns=column_names)

In [ ]:
# creating column list for insertion for person table
cols = "`,`".join([str(i) for i in df_labevents_mimic_coded.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df_labevents_mimic_coded.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    sql = "INSERT INTO `obs` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    try:
        openmrscursor.execute(sql1)
        openmrscursor.execute(sql, tuple(row))
        openmrscursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit() 